<a href="https://colab.research.google.com/github/inshra12/ESM-MLP-baseline-protein-function/blob/main/notebooks/model2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Colab cell (code)
!pip install biopython scipy scikit-learn fair-esm tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 3.4 MB/s eta 0:00:00


In [4]:
# If you uploaded alphafold_pdb directly to Colab VM, set:
PDB_DIR = "/content/alphafold_pdb"

In [5]:
# Paths to your existing files (update if located in Drive)
CLS_EMBEDDINGS = "/content/cls_embeddings.csv"          # ESM CLS embeddings (ID + 1280 cols)
LABELS_CSV      = "/content/esm_with_multihot_labels.csv"  # multi-hot labels (ID + GO cols)

In [8]:
STATIC_OUT      = "/content/static_features.csv"

In [6]:
from structure_embedding import extract_embeddings_from_folder

In [9]:
pdb_dir = PDB_DIR
out_csv = STATIC_OUT
extract_embeddings_from_folder(pdb_dir, output_csv=out_csv, hist_bins=10, contact_cutoff=8.0)



✅ O00222 -> features len 18
✅ O14842 -> features len 18
✅ O15303 -> features len 18
✅ O43603 -> features len 18
✅ O43613 -> features len 18
✅ O43614 -> features len 18
✅ O60353 -> features len 18
✅ O60755 -> features len 18
✅ O70526 -> features len 18
✅ O70528 -> features len 18
✅ O75084 -> features len 18
✅ O75899 -> features len 18
✅ O95136 -> features len 18
✅ O95665 -> features len 18
✅ O95977 -> features len 18
✅ P01019 -> features len 18
✅ P01275 -> features len 18
✅ P02699 -> features len 18
✅ P05305 -> features len 18
✅ P06199 -> features len 18
✅ P07550 -> features len 18
✅ P07700 -> features len 18
✅ P08172 -> features len 18
✅ P08173 -> features len 18
✅ P08482 -> features len 18
✅ P08483 -> features len 18
✅ P08485 -> features len 18
✅ P08588 -> features len 18
✅ P08908 -> features len 18
✅ P08909 -> features len 18
✅ P08911 -> features len 18
✅ P08912 -> features len 18
✅ P08913 -> features len 18
✅ P0DMS8 -> features len 18
✅ P10980 -> features len 18
✅ P11229 -> features

'/content/static_features.csv'

In [10]:
# Colab cell
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# Paths (update if stored in Drive)
cls_path = CLS_EMBEDDINGS
static_path = STATIC_OUT
labels_path = LABELS_CSV

esm_df = pd.read_csv(cls_path)      # ID + 1280 ESM columns (0..1279)
static_df = pd.read_csv(static_path) # ID + structural features
labels_df = pd.read_csv(labels_path) # ID + GO columns

# Clean whitespace in column names
labels_df.columns = [c.strip() for c in labels_df.columns]
esm_df.columns = [c.strip() for c in esm_df.columns]
static_df.columns = [c.strip() for c in static_df.columns]

# Merge: first join esm + static on ID, then merge labels
merged = pd.merge(esm_df, static_df, on="ID", how="inner")
merged = pd.merge(merged, labels_df, on="ID", how="inner")

print("Merged shape:", merged.shape)
# Save merged file for record
merged.to_csv("esm_static_with_labels.csv", index=False)
print("Saved: esm_static_with_labels.csv")


Merged shape: (265, 2593)
Saved: esm_static_with_labels.csv


In [16]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# ==== Paths to your files ====
CLS_EMB_PATH   = "/content/cls_embeddings.csv"      # ESM CLS embedding file
STATIC_PATH    = "/content/static_features.csv"     # Static structure embedding file
OUTPUT_PATH    = "/content/esm_static.csv"          # Output merged file (no labels)

# ==== Load the two CSVs ====
cls_df = pd.read_csv(CLS_EMB_PATH)    # Expected: ID, 0, 1, 2, ..., 1279
static_df = pd.read_csv(STATIC_PATH)  # Expected: ID + static feature columns

print("CLS shape:", cls_df.shape)
print("Static shape:", static_df.shape)

# ==== Merge by ID ====
merged_df = pd.merge(cls_df, static_df, on="ID", how="inner")
print("Merged shape (before scaling):", merged_df.shape)

# ==== Identify column groups ====
esm_cols = [c for c in cls_df.columns if c != "ID"]  # 1280 cols
static_cols = [c for c in static_df.columns if c != "ID"]

# ==== Scale static features ====
scaler = StandardScaler()
scaled_static = scaler.fit_transform(merged_df[static_cols])

# ==== Combine ESM + scaled static ====
esm_values = merged_df[esm_cols].values
X_combined = np.hstack([esm_values, scaled_static])

# ==== Create final DataFrame ====
final_df = pd.DataFrame(X_combined, columns=list(esm_cols) + static_cols)
final_df.insert(0, "ID", merged_df["ID"])

# ==== Save without labels ====
final_df.to_csv(OUTPUT_PATH, index=False)
print(f"Saved merged embeddings to {OUTPUT_PATH}")


CLS shape: (265, 1281)
Static shape: (265, 19)
Merged shape (before scaling): (265, 1299)
Saved merged embeddings to /content/esm_static.csv


In [11]:
# Colab cell
from sklearn.model_selection import train_test_split

df = pd.read_csv("esm_static_with_labels.csv")

# Identify GO columns: assume they start after embedding+static columns -- but safer:
# We will detect GO columns by 'GO:' prefix in column names
go_cols = [c for c in df.columns if c.startswith("GO:")]
print("Number of GO labels:", len(go_cols))

# Identify ESM embedding columns (common naming 0..1279)
esm_cols = [c for c in df.columns if c.isdigit()]  # if ESM columns are numeric strings
# If not numeric, adjust accordingly (e.g., '0','1',..)
print("ESM cols count:", len(esm_cols))

# Static feature columns = all columns between 'ID' and GO columns not in esm_cols
static_cols = [c for c in df.columns if c not in (["ID"] + esm_cols + go_cols)]
print("Static cols:", static_cols)

# Build X and Y
X = df[esm_cols + static_cols].copy()
Y = df[go_cols].copy()

# Scale static features (so they play nicely with ESM embeddings)
scaler = StandardScaler()
X_static = scaler.fit_transform(X[static_cols])
X_scaled = np.hstack([X[esm_cols].values, X_static])
X_df = pd.DataFrame(X_scaled, columns = list(esm_cols) + static_cols)
X_df.insert(0, "ID", df["ID"])
merged_ready = pd.concat([X_df, Y.reset_index(drop=True)], axis=1)

# Quick train/test split (same as Model 1 style)
X_train, X_test, Y_train, Y_test = train_test_split(
    X_scaled, Y.values, test_size=0.2, random_state=42
)

print("Train samples:", X_train.shape[0], "Test:", X_test.shape[0])


Number of GO labels: 14
ESM cols count: 0
Static cols: ['0_x', '1_x', '2_x', '3_x', '4_x', '5_x', '6_x', '7_x', '8_x', '9_x', '10_x', '11_x', '12_x', '13_x', '14_x', '15_x', '16_x', '17_x', '18_x', '19_x', '20_x', '21_x', '22_x', '23_x', '24_x', '25_x', '26_x', '27_x', '28_x', '29_x', '30_x', '31_x', '32_x', '33_x', '34_x', '35_x', '36_x', '37_x', '38_x', '39_x', '40_x', '41_x', '42_x', '43_x', '44_x', '45_x', '46_x', '47_x', '48_x', '49_x', '50_x', '51_x', '52_x', '53_x', '54_x', '55_x', '56_x', '57_x', '58_x', '59_x', '60_x', '61_x', '62_x', '63_x', '64_x', '65_x', '66_x', '67_x', '68_x', '69_x', '70_x', '71_x', '72_x', '73_x', '74_x', '75_x', '76_x', '77_x', '78_x', '79_x', '80_x', '81_x', '82_x', '83_x', '84_x', '85_x', '86_x', '87_x', '88_x', '89_x', '90_x', '91_x', '92_x', '93_x', '94_x', '95_x', '96_x', '97_x', '98_x', '99_x', '100_x', '101_x', '102_x', '103_x', '104_x', '105_x', '106_x', '107_x', '108_x', '109_x', '110_x', '111_x', '112_x', '113_x', '114_x', '115_x', '116_x', '

In [14]:
from MLP_Model import build_mlp_model
from train_MLP import train_model

In [17]:
X = pd.read_csv("/content/esm_static.csv").iloc[:, 1:].values  # skip ID col
Y = pd.read_csv("/content/esm_with_multihot_labels.csv").values

input_size = X.shape[1]
output_size = Y.shape[1]

In [18]:
input_size

1298

In [19]:
output_size

1295